# Chapter 6

# Data Loading, Storage, and File Formats

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv('/Users/allenc/PyCharmProjects/JupyterProjects/WesWrangling/examples/ex1.csv')

pandas features a number of functions for reading tabular data as a DataFrame object. Table 6-1 summarizes some of them, though read_csv and read_table are likely the ones you’ll use the most.
I’ll give an overview of the mechanics of these functions, which are meant to convert text data into a DataFrame. The optional arguments for these functions may fall into a few categories:

Indexing
Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.

Type inference and data conversion

This includes the user-defined value conversions and custom list of missing value markers.
Datetime parsing

Includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.

Iterating
Support for iterating over chunks of very large files.

Unclean data issues
Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Here I used the Unix cat shell command to print the raw contents of the file to the screen. If you’re on Windows, you can use type instead of cat to achieve the same effect.

In [4]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
# Since this is comma-delimited, we can use read_csv to read it into a DataFrame:
df = pd.read_csv('examples/ex1.csv')

In [6]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


We could also have used read_table and specified the delimiter:

In [8]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
# A file will not always have a header row. Consider this file:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:

In [10]:
pd.read_csv('examples/ex2.csv', header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
pd.read_csv('examples/ex2.csv', names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named 'message' using the index_col argument:

In [12]:
names = ['a', 'b', 'c', 'd', 'message']

In [13]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [14]:
# In the event that you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [15]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                             index_col=['key1', 'key2'])

In [16]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

In [17]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

While you could do some munging by hand, the fields here are separated by a vari‐ able amount of whitespace. In these cases, you can pass a regular expression as a delimiter for read_table. This can be expressed by the regular expression \s+, so we have then:

In [18]:
result = pd.read_table('examples/ex3.txt', sep='\s+')

In [19]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


The parser functions have many additional arguments to help you handle the wide variety of exception file formats that occur (see a partial listing in Table 6-2). For example, you can skip the first, third, and fourth rows of a file with skiprows:

In [20]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [21]:
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file pars‐ ing process. Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as NA and NULL:

In [22]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [23]:
result = pd.read_csv('examples/ex5.csv')

In [24]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [25]:
# The na_values option can take either a list or set of strings to consider missing values:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])

In [26]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
# Different NA sentinels can be specified for each column in a dict:
sentinels = {'message':['foo', 'NA'], 'something':['two']}

In [28]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


# Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to cor‐ rectly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file.
Before we look at a large file, we make the pandas display settings more compact:

In [29]:
pd.options.display.max_rows=10

In [30]:
result = pd.read_csv('examples/ex6.csv')

In [31]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [32]:
# If you want to only read a small number of rows (avoiding reading the entire file), specify that with nrows:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [33]:
# To read a file in pieces, specify a chunksize as a number of rows:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [34]:
chunker

The TextParser object returned by read_csv allows you to iterate over the parts of the file according to the chunksize. For example, we can iterate over ex6.csv, aggre‐ gating the value counts in the 'key' column like so:

In [35]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([]) 
for piece in chunker:
        tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

/var/folders/vm/hn4f87jj1kv93ht83n76p_d00000gp/T/ipykernel_1798/919391867.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [36]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

# Writing Data to Text Format

In [37]:
# Data can also be exported to a delimited format. Let’s consider one of the CSV files read before:
data = pd.read_csv('examples/ex5.csv')

In [38]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [39]:
# Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:
data.to_csv('examples/out.csv')

In [40]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [41]:
# Other delimiters can be used, of course (writing to sys.stdout so it prints the text result to the console):
import sys

In [42]:
data.to_csv(sys.stdout, sep='!')

!something!a!b!c!d!message
0!one!1!2!3.0!4!
1!two!5!6!!8!world
2!three!9!10!11.0!12!foo


In [43]:
# Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:
data.to_csv(sys.stdout,na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [44]:
# With no other options specified, both the row and column labels are written. Both of these can be disabled:
data.to_csv(sys.stdout,index=False,header= False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [45]:
data.to_csv(sys.stdout, index=False, columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [46]:
# Series also has a to_csv method:
dates = pd.date_range('1/1/2000', periods=7)

In [47]:
ts = pd.Series(np.arange(7), index=dates)

In [48]:
ts.to_csv('examples/tseries.csv')

In [49]:
!cat examples/tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


# Working with Delimited Formats

In [50]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


For any file with a single-character delimiter, you can use Python’s built-in csv mod‐ ule. To use it, pass any open file or file-like object to csv.reader:

In [51]:
import csv

In [52]:
f = open('examples/ex7.csv')

In [53]:
reader = csv.reader(f)

In [54]:
Iterating through the reader like a file yields tuples of values with any quote charac‐
ters removed:

SyntaxError: invalid syntax (1551310004.py, line 1)

In [55]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


From there, it’s up to you to do the wrangling necessary to put the data in the form that you need it. Let’s take this step by step. First, we read the file into a list of lines:

In [56]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [57]:
# Then, we split the lines into the header line and the data lines:
header, values = lines[0], lines[1:]

Then we can create a dictionary of data columns using a dictionary comprehension
and the expression zip(*values), which transposes rows to columns:

In [58]:
data_dict = {h:v for h, v in zip(header, zip(*values))}

In [59]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different flavors. To define a new format with a different delimiter, string quoting convention, or line terminator, we define a simple subclass of csv.Dialect:

In [60]:
class my_dialect(csv.Dialect): 
    lineterminator = '\n' 
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [61]:
reader = csv.reader(f, dialect=my_dialect)

ValueError: I/O operation on closed file.

In [65]:
# We can also give individual CSV dialect parameters as keywords to csv.reader without having to define a subclass:
reader = csv.reader(f, delimiter='|')

ValueError: I/O operation on closed file.

In [66]:
# To write delimited files manually, you can use csv.writer. It accepts an open, writa‐ ble file object and the same dialect and format options as csv.reader:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect) 
    writer.writerow(('one', 'two', 'three')) 
    writer.writerow(('1', '2', '3')) 
    writer.writerow(('4', '5', '6')) 
    writer.writerow(('7', '8', '9'))

# JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example:

There are several Python libraries for reading and writing JSON data. I’ll use json here, as it is built into the Python standard library. To convert a JSON string to Python form, use json.loads:

In [67]:
 obj = """
    {"name": "Wes",
     "places_lived": ["United States", "Spain", "Germany"],
     "pet": null,
     "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
                  {"name": "Katie", "age": 38,
                   "pets": ["Sixes", "Stache", "Cisco"]}]
} """

In [68]:
import json

In [69]:
result = json.loads(obj)

In [70]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [71]:
# json.dumps, on the other hand, converts a Python object back to JSON:
asjson = json.dumps(result)

How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dicts (which were previously JSON objects) to the DataFrame constructor and select a sub‐ set of the data fields:

In [72]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])

In [73]:
siblings

,name,age
0,Scott,30
1,Katie,38


The pandas.read_json can automatically convert JSON datasets in specific arrange‐ ments into a Series or DataFrame. For example:

In [74]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [75]:
# The default options for pandas.read_json assume that each object in the JSON array is a row in the table:
data = pd.read_json('examples/example.json')

In [76]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [77]:
# If you need to export data from pandas to JSON, one way is to use the to_json meth‐ ods on Series and DataFrame:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [78]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


# XML and HTML: Web Scraping

In [79]:
# !pip3 install lxml
# !pip3 install beautifulsoup4 html5lib

The pandas.read_html function has a number of options, but by default it searches for and attempts to parse all tabular data contained within <table> tags. The result is a list of DataFrame objects:

In [80]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [81]:
len(tables)

1

In [82]:
failures = tables[0]

In [83]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [84]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [85]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

In [86]:
# Using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot:
from lxml import objectify

In [87]:
path = 'examples/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

FileNotFoundError: [Errno 2] No such file or directory: 'examples/mta_perf/Performance_MNR.xml'

In [88]:
data = []

In [89]:
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
                   'DESIRED_CHANGE', 'DECIMAL_PLACES']

In [90]:
for elt in root.INDICATOR: 
    el_data = {}
    for child in elt.getchildren(): 
        if child.tag in skip_fields:
            continue
    el_data[child.tag] = child.pyval
data.append(el_data)

NameError: name 'root' is not defined

In [91]:
# Lastly, convert this list of dicts into a DataFrame:
perf = pd.DataFrame(data)

In [92]:
perf.head()

""


XML data can get much more complicated than this example. Each tag can have metadata, too. Consider an HTML link tag, which is also valid XML:

In [93]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>' 
root = objectify.parse(StringIO(tag)).getroot()

In [94]:
# You can now access any of the fields (like href) in the tag or the link text:
root

<Element a at 0x11a27c180>

In [95]:
root.get('href')

'http://www.google.com'

In [96]:
root.text

'Google'

# 6.2 Binary Data Formats

One of the easiest ways to store data (also known as serialization) efficiently in binary format is using Python’s built-in pickle serialization. pandas objects all have a to_pickle method that writes the data to disk in pickle format:

In [97]:
frame = pd.read_csv('examples/ex1.csv')

In [98]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [99]:
frame.to_pickle('examples/frame_pickle')

You can read any “pickled” object stored in a file by using the built-in pickle directly,
or even more conveniently using pandas.read_pickle:

In [100]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


pickle is only recommended as a short-term storage format. The problem is that it is hard to guarantee that the format will be stable over time; an object pickled today may not unpickle with a later version of a library. We have tried to maintain backward compati‐ bility when possible, but at some point in the future it may be nec‐ essary to “break” the pickle format.

# Using HDF5 Format

HDF5 is a well-regarded file format intended for storing large quantities of scientific array data. The HDFStore class works like a dict and handles the low-level details:

In [101]:
# !pip3 install tables

In [102]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [103]:
store = pd.HDFStore('mydata.h5')

In [104]:
store['obj1'] = frame

In [105]:
store['obj1_col'] = frame['a']

In [106]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [107]:
# Objects contained in the HDF5 file can then be retrieved with the same dict-like API:
store['obj1']

,a
0,-0.318429
1,-0.699373
2,0.054161
3,0.782822
4,-0.391584
...,...
95,-0.833165
96,0.605798
97,0.885268
98,-1.145627


HDFStore supports two storage schemas, 'fixed' and 'table'. The latter is generally slower, but it supports query operations using a special syntax:

In [108]:
store.put('obj2', frame, format='table')

In [109]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-1.888172
11,1.092120
12,0.107455
13,-1.947336
14,0.310381
15,-0.816792


In [110]:
store.close()

In [111]:
# The pandas.read_hdf function gives you a shortcut to these tools:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [112]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.318429
1,-0.699373
2,0.054161
3,0.782822
4,-0.391584


In [113]:
#!pip3 install openpyxl

In [114]:
# To use ExcelFile, create an instance by passing a path to an xls or xlsx file:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [115]:
# Data stored in a sheet can then be read into DataFrame with parse:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


If you are reading multiple sheets in a file, then it is faster to create the ExcelFile, but you can also simply pass the filename to pandas.read_excel:

In [116]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

In [117]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


To write pandas data to Excel format, you must first create an ExcelWriter, then write data to it using pandas objects’ to_excel method:

In [118]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

In [119]:
frame.to_excel(writer, 'Sheet1')

In [120]:
writer.save()

In [121]:
# You can also pass a file path to to_excel and avoid the ExcelWriter:
frame.to_excel('examples/ex2.xlsx')

# 6.3 Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one easy-to-use method that I recommend is the requests package.

To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library:

In [122]:
import requests

In [123]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [128]:
resp = requests.get(url)

In [129]:
resp

<Response [200]>

In [130]:
# The Response object’s json method will return a dictionary containing JSON parsed into native Python objects:
data = resp.json()

In [131]:
data[0]['title']

'CI/TST: Make test_vector_resize more deterministic'

Each element in data is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass data directly to DataFrame and extract fields of interest:

In [132]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                         'labels', 'state'])

In [133]:
issues

,number,title,labels,state
0,46602,CI/TST: Make test_vector_resize more determini...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
1,46601,WIP/BUG: algorithms.factorize moves null value...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,46600,CI: Increase GitHub Actions resources,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
3,46599,CI: Simplify call to asv,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
4,46598,CI: Add durations to benchmarks build,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
...,...,...,...,...
25,46573,BUG: prod and groupby.prod behave differently,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,46571,ENH: Allow usage of custom library to serializ...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
27,46570,POC For docker compose,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
28,46569,BUG: ENH: Allow callable for on_bad_lines in ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


# 6.4 Interacting with Databases

Loading data from SQL into a DataFrame is fairly straightforward, and pandas has some functions to simplify the process. As an example, I’ll create a SQLite database using Python’s built-in sqlite3 driver:

In [134]:
import sqlite3

In [135]:
query = """
        CREATE TABLE test
        (a VARCHAR(20), b VARCHAR(20),
        c REAL,        d INTEGER
        );"""

In [136]:
con = sqlite3.connect('mydata.sqlite')

In [137]:
con.execute(query)

In [138]:
con.commit()

Then, insert a few rows of data:

In [140]:
data = [('Atlanta', 'Georgia', 1.25, 6),
                ('Tallahassee', 'Florida', 2.6, 3),
                ('Sacramento', 'California', 1.7, 5)]

In [141]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [142]:
con.executemany(stmt, data)

In [143]:
con.commit()

Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list of tuples when selecting data from a table:

In [144]:
cursor = con.execute('select * from test')

In [145]:
rows = cursor.fetchall()

In [146]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

You can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s description attribute:

In [147]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [148]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [150]:
!pip3 install sqlalchemy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 KB 6.9 MB/s eta 0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme wi

In [151]:
import sqlalchemy as sqla

In [152]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [153]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


# 6.5 Conclusion

Getting access to data is frequently the first step in the data analysis process. We have looked at a number of useful tools in this chapter that should help you get started. In the upcoming chapters we will dig deeper into data wrangling, data visualization, time series analysis, and other topics.